In [55]:
import json
 
# Opening JSON file
f = open('../Data/Data.json',)
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [56]:
from collections import OrderedDict


In [57]:
sorted_data=[]
for element in data:
    sentence_votes=element["Top level comments"]
    d_sorted_by_value = OrderedDict(sorted(element["Top level comments"].items(), key=lambda x: x[1]))
    sorted_data.append(d_sorted_by_value)

In [58]:
final_dataset = []

for i in sorted_data:
    dataset=[]
    seen_sentence1 = []
    for sentence1 in i.keys():
        seen_sentence1.append(sentence1)
        for sentence2 in i.keys():
            if sentence1 == sentence2 or sentence2 not in seen_sentence1:
                continue
            if sentence1 == '[removed]' or sentence2 == '[removed]':
                continue
            dataset.append([sentence1, sentence2, i[sentence1]-i[sentence2]])
    final_dataset.append(dataset)   

In [59]:
final_dataset = []

for i in sorted_data:
    dataset=[]
    seen_sentence1 = []
    for sentence1 in i.keys():
        seen_sentence1.append(sentence1)
        for sentence2 in i.keys():
            if sentence1 == sentence2 or sentence2 not in seen_sentence1:
                continue
            if sentence1 == '[removed]' or sentence2 == '[removed]':
                continue
            dataset.append([sentence1, sentence2, i[sentence1]-i[sentence2]])
    final_dataset.append(dataset)   

In [60]:
pop_ind = []
for ind, data in enumerate(final_dataset):
    if len(data) == 0:
        pop_ind.append(ind)

In [61]:
for i in pop_ind:
    final_dataset.pop(i)

In [62]:
len(final_dataset[9])

45

In [63]:
import itertools


In [64]:
final_dataset1 = list(itertools.chain(*final_dataset))


In [65]:
file_path = "difference_votes.csv"

In [66]:
import csv
with open(file_path, 'a') as outcsv:   
    #configure writer to write standard csv file
    writer = csv.writer(outcsv, delimiter=',')
    writer.writerow(['sentence1', 'sentence2', 'difference_votes'])
    for item in final_dataset1:
        #Write item to outcsv
        writer.writerow([item[0], item[1], item[2]])


In [67]:
import pandas as pd

In [68]:
df = pd.read_csv("difference_votes.csv")

In [85]:
df

,sentence1,sentence2,difference_votes
0,One person I knew who was allergic to gluten s...,We are locking this thread because of the high...,11
1,a little off topic : one thing I read a long t...,We are locking this thread because of the high...,383
2,a little off topic : one thing I read a long t...,One person I knew who was allergic to gluten s...,372
3,Probably the same reason people are lactose in...,We are locking this thread because of the high...,456
4,Probably the same reason people are lactose in...,One person I knew who was allergic to gluten s...,445
...,...,...,...
74986,Oh man the first ELI5 I can answer! I'm a past...,The increasing inclusion of dairy fat to make ...,10409
74987,Oh man the first ELI5 I can answer! I'm a past...,I'm a bean to bar chocolate maker in the US.\n...,10401
74988,Oh man the first ELI5 I can answer! I'm a past...,There are actually legal definitions set out b...,10398
74989,Oh man the first ELI5 I can answer! I'm a past...,The type of chocolate depends on the sugar to ...,10337


In [84]:
df_test

In [83]:
df_test = df.drop(df.index[df['difference_votes'] == 11], inplace = True)

In [231]:
'''This method is used to enforce proper spacing
        Ex: In the data, you may have '[the image](https://image.xyz)';
        this method creates space between alt text ("the image") and the URL.'''
        
df["sentence1"] = df["sentence1"].str.replace('\[|\]', ' ', regex=True)
df["sentence2"] = df["sentence2"].str.replace('\[|\]', ' ', regex=True)

In [233]:
# removeURL
df["sentence1"] = df["sentence1"].str.replace('\(http\S+', 'URL', regex=True)
df["sentence2"] = df["sentence2"].str.replace('\(http\S+', 'URL', regex=True)

In [234]:
# removeSymbols
df["sentence1"] = df["sentence1"].str.replace('/r/', '', regex=True)
df["sentence2"] = df["sentence2"].str.replace('/r/', '', regex=True)

df["sentence1"] = df["sentence1"].str.replace('[^\.\'A-Za-z0-9]+', ' ', regex=True)
df["sentence2"] = df["sentence2"].str.replace('[^\.\'A-Za-z0-9]+', ' ', regex=True)

# removeNumbers
df["sentence1"] = df["sentence1"].str.replace('\S*\d\S*', '', regex=True)
df["sentence2"] = df["sentence2"].str.replace('\S*\d\S*', '', regex=True)




In [256]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test


In [260]:
train, validate, test = train_validate_test_split(df)

In [5]:
import numpy as np
import pandas as pd

In [1]:
train.to_csv("train_regressor.csv",index=False)

NameError: name 'train' is not defined

In [2]:
test.to_csv("test_regressor.csv", index=False)

NameError: name 'test' is not defined

In [268]:
validate.to_csv("validate_regressor.csv", index=False)

In [107]:
train = pd.read_csv("train_regressor.csv")
dev = pd.read_csv('validate_regressor.csv')
test = pd.read_csv("test_regressor.csv")


In [108]:
type(train["difference_votes"])

pandas.core.series.Series

In [109]:
train['difference_votes'] = np.where(train['difference_votes']< 200 , 0, train['difference_votes'])


In [110]:
train['difference_votes'] = np.where(train['difference_votes']>= 200 , 1, train['difference_votes'])


In [114]:
test['difference_votes'] = np.where(test['difference_votes']< 200 , 0, test['difference_votes'])
test['difference_votes'] = np.where(test['difference_votes']>= 200 , 1, test['difference_votes'])

In [115]:
dev['difference_votes'] = np.where(dev['difference_votes']< 200 , 0, dev['difference_votes'])
dev['difference_votes'] = np.where(dev['difference_votes']>= 200 , 1, dev['difference_votes'])

In [117]:
train.to_csv("finaltrain_regressor.csv",index=False)
test.to_csv("finaltest_regressor.csv", index=False)
dev.to_csv("finalvalidate_regressor.csv", index=False)

In [6]:
def load_reddit_data():

    train = pd.read_csv("train_regressor.csv")
    dev = pd.read_csv('validate_regressor.csv')
    test = pd.read_csv("test_regressor.csv")
    def yielder():
        for partition in [train, dev, test]:
            data = []
            for idx,_ in enumerate(partition):
                data.append({
                    'index': idx,
                    'sentence_1': partition["sentence1"][idx],
                    'sentence_2': partition["sentence2"][idx],
                    'similarity': float(partition["difference_votes"][idx])
                })
            yield data

    return tuple([dataset for dataset in yielder()])

In [7]:
train = pd.read_csv("train_regressor.csv")

Generating Dummy Dataset: 100%|██████████| 10000/10000 [00:03<00:00, 2500.20it/s]


In [88]:
import tqdm

In [91]:
type(corpus)

list